# 트랜스포머로 만드는 대화형 챗봇 프로젝트 🌠

## Transformer 대비 변경이 필요한 부분

**1. 모델 구조**
- 기존 프로젝트 : Encoder + Decoder 두 블록으로 구성되어 있습니다.
- 변경 내역 : Decoder-only 구조로, Encoder 제거한 다음 Decoder 블록만 사용했습니다. Self-Attention에 미래 토큰 마스킹 추가도 진행했습니다.

**2. 입력 포맷**
- 기존 프로젝트 : "USER:", "ASSISTANT:" 접두사를 문자열 그대로 붙여 SentencePiece로 토큰화했습니다.
- 변경 내역 : 특수 역할 토큰 [USR], [BOT], [DELIM]을 명시적으로 부여하여, 입력을 [BOS][USR] 질문 [DELIM][BOT] 답변 [DELIM] … [EOS] 형태로 구성했습니다.

**3. 위치 정보(Positional Encoding)**
- 기존 프로젝트 : Transformer 논문과 동일한 sinusoidal positional encoding 사용했습니다.
- 변경 내역 : 학습 가능한 positional embedding(pos_emb)을 추가하여 토큰 임베딩과 합산했습니다.

**4. 학습 목표**
- 기존 프로젝트 : Encoder 입력(질문) → Decoder 출력(답변) 방식으로 sequence-to-sequence 학습이 목표였습니다.
- 변경 내역 : 입력 전체 시퀀스에서 다음 토큰을 예측하도록 변경했습니다.

**5. 손실 계산 방식**
- 기존 프로젝트 : Decoder 출력 전체에 대해 손실 계산했습니다.
- 변경 내역 : 사용자 발화 부분은 마스킹하여 손실에서 제외하고, 답변 토큰만 손실 계산하도록 했습니다.

**6. 출력 생성**
- 기존 프로젝트 : Greedy Search 또는 단순 샘플링 기반입니다.
- 변경 내역 : 샘플링 강화 후에 temperature, top-p(nucleus sampling), repetition penalty, stop tokens을 적용했습니다.

**7. 파라미터 공유**
- 기존 프로젝트 : 입력 임베딩과 출력 레이어(Softmax)가 별개였습니다.
- 변경 내역 : 임베딩 ↔ LM Head weight tying 적용했습니다.

### 1. 라이브러리 불러오기

- ```SentencePiece``` : 텍스트 토큰화 라이브러리로, 특히 한국어와 일본어 같이 띄어쓰기 구분이 애매한 언어에서 자주 사용됩니다.

In [1]:
!pip install -q torch sentencepiece

In [2]:
# 패키지 임포트

import os, math, time, random
from dataclasses import dataclass
from typing import List, Tuple

import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

In [3]:
# GPU를 사용할 수 있다면?
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### 2. 데이터 전처리하기

[입력 구성 방식]

1. 기존 프로젝트
- CSV의 Q, A를 불러와서 각각 Encoder 입력(질문), Decoder 목표(답변)으로 분리했습니다.
- 전처리 단계에서 질문 시퀀스, 답변 시퀀스를 따로 만들어서 DataLoader에 넣었습니다.
- 입력 포맷은 "USER:" + Q, "ASSISTANT:" + A 같은 문자열 접두사 기반이었습니다.
- SentencePiece로 subword 단위 토큰화 → Encoder/Decoder 각각 padding을 적용했습니다.

2. 변경 내역
- Encoder가 없으므로 Q, A를 하나의 시퀀스로 이어 붙였습니다.
- ```[BOS][USR] Q [DELIM][BOT] A [DELIM] … [EOS]``` 형태로 구성됩니다.
- ```DialogDataset``` 클래스에서 turn 단위로 묶어서 block size 기준으로 슬라이싱합니다.
- x와 y는 단일 시퀀스에서 shifted LM 방식으로 생성합니다.

In [4]:
pwd

'/home/jovyan/work/0818'

In [5]:
# 데이터셋 경로 지정
CSV_PATH = "./data/ChatbotData.csv"

# 데이터셋 불러오기
df = pd.read_csv(CSV_PATH)

# 데이터 프레임에 Q/A가 들어와 확인해주는 용도
assert "Q" in df.columns and "A" in df.columns

df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [6]:
# 기본 전처리 - 결측치와 중복값 제거

df = df.dropna(subset=["Q","A"]).copy()
df["Q"] = df["Q"].astype(str).str.strip()
df["A"] = df["A"].astype(str).str.strip()
df = df.drop_duplicates(subset=["Q","A"]).reset_index(drop=True)

print(df.shape)
df.head(3)

(11750, 3)


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0


In [7]:
# train / valid 분할
train_df, val_df = train_test_split(df, test_size=0.05, random_state=42, shuffle=True)

# 대화를 묶는 함수 - [(질문, 답변)] 형태로 반환
def df_to_dialogs(df, turns_per_dialog=2):
    pairs = list(zip(df["Q"].tolist(), df["A"].tolist()))
    dialogs, buf = [], []
    for q,a in pairs:
        buf.append((q,a))
        if len(buf) == turns_per_dialog:
            dialogs.append(buf); buf = []
    if buf: dialogs.append(buf)
    return dialogs

TURNS_PER_DIALOG = 2
dialogs_train = df_to_dialogs(train_df, TURNS_PER_DIALOG)
dialogs_val   = df_to_dialogs(val_df,   TURNS_PER_DIALOG)
len(dialogs_train), len(dialogs_val)

(5581, 294)

[특수 토큰 처리]

1. 기존 프로젝트
- USER:, ASSISTANT:는 그냥 문자열이라 SentencePiece에서 여러 subword로 분리했었습니다. (학습 시 의미 혼동 발생 가능성이 있다고 합니다..)
- 위치 구분은 문자열 패턴에 의존하는 방식이었습니다.

2. 변경 내역
- [USR], [BOT], [DELIM]을 단일 토큰으로 추가했습니다.
- 역할과 턴 경계를 명확하게 하여 모델이 학습하기 쉬워집니다.
- 디코딩 시에도 이 토큰들을 제거하고 자연스러운 문장만 출력하도록 했습니다.

In [8]:
import sentencepiece as spm

def build_tokenizer(spm_path="spm_ko.model", prefer_sp=True):
    if prefer_sp and os.path.exists(spm_path):
        # SentencePiece 가져오기
        sp = spm.SentencePieceProcessor()
        sp.load(spm_path)
        sp_vocab = sp.get_piece_size()

        # SentencePiece에 특수 토큰이 없다면 추가 ID를 뒤쪽에 할당
        PAD = sp.pad_id() if sp.pad_id() >= 0 else sp_vocab; sp_vocab += (sp.pad_id()<0)
        UNK = sp.unk_id() if sp.unk_id() >= 0 else sp_vocab; sp_vocab += (sp.unk_id()<0)
        BOS = sp.bos_id() if sp.bos_id() >= 0 else sp_vocab; sp_vocab += (sp.bos_id()<0)
        EOS = sp.eos_id() if sp.eos_id() >= 0 else sp_vocab; sp_vocab += (sp.eos_id()<0)

        # 턴 구분자 처리
        DELIM = sp.piece_to_id("<DELIM>") if "<DELIM>" in [sp.id_to_piece(i) for i in range(sp.get_piece_size())] else sp_vocab; sp_vocab += ("<DELIM>" not in [sp.id_to_piece(i) for i in range(sp.get_piece_size())])

        # 역할 토큰 경계 명확히 하기 (짤리지 않게)
        USR, BOT = sp_vocab, sp_vocab+1
        sp_vocab += 2

        class SPTokenizer:
            def __init__(self, sp):
                self.sp=sp; self.PAD=PAD; self.UNK=UNK; self.BOS=BOS; self.EOS=EOS
                self.DELIM=DELIM; self.USR=USR; self.BOT=BOT; self.vocab_size=sp_vocab
            def encode_text(self,s,add_special=True):
                ids=self.sp.encode(s,out_type=int)
                return [self.BOS]+ids+[self.EOS] if add_special else ids
            def join_with_delim(self,parts):
                seq=[]
                for i,p in enumerate(parts):
                    if i>0: seq.append(self.DELIM)
                    seq.extend(p)
                return seq
            def decode(self,ids):
                drop={self.PAD,self.UNK,self.BOS,self.EOS,self.DELIM,self.USR,self.BOT}
                keep=[i for i in ids if (i not in drop)]
                return self.sp.decode(keep)
        print(f"Loaded SentencePiece tokenizer, vocab={sp_vocab}")
        return SPTokenizer(sp)

tok = build_tokenizer("spm_ko.model", prefer_sp=True)

Loaded SentencePiece tokenizer, vocab=4003


[손실 계산용 마스크]

1. 기존 프로젝트
- Decoder의 모든 토큰(질문 포함)에 대해 손실을 계산했었습니다.
- Q와 A를 명확히 분리했었기에 딱히 문제가 없었습니다.

2. 변경 내역
- 단일 시퀀스 안에서 사용자 발화 부분은 마스킹하여 loss에서 제외했습니다.
- 답변 토큰 예측에만 집중할 수 있도록 ```mask_user_loss=True``` 옵션을 추가했습니다.

In [9]:
class DialogDataset(Dataset):

    """
    입력 시퀀스 구성 :
    [BOS] [USR] user_ids [DELIM] [BOT] answer_ids [DELIM] ... [EOS]
    
    학습 타깃(y)은 x를 한 칸 오른쪽으로 민 형태(next-token 예측).
    mask_user_loss=True이면 사용자 구간은 loss에서 제외함(PAD로 덮어써 ignore_index 처리).
    """
    
    def __init__(self, dialogs, tokenizer, block_size:int, mask_user_loss=True):
        self.tok=tokenizer; self.block_size=block_size; self.mask_user_loss=mask_user_loss
        self.samples=[]; self.masks=[]
        for dialog in dialogs:
            ids=[self.tok.BOS]; keep=[False]
            for (u,a) in dialog:
                u_ids=self.tok.encode_text(u,add_special=False)
                a_ids=self.tok.encode_text(a,add_special=False)
                ids+=[self.tok.USR]+u_ids+[self.tok.DELIM]+[self.tok.BOT]+a_ids+[self.tok.DELIM]
                keep+=[False]+([False]*len(u_ids) if mask_user_loss else [True]*len(u_ids))+[False]+[True]+[True]*len(a_ids)+[False]
            ids+=[self.tok.EOS]; keep+=[False]
            for i in range(0,len(ids)-1,self.block_size):
                chunk=ids[i:i+self.block_size+1]; mchunk=keep[i:i+self.block_size+1]
                if len(chunk)<2: break
                self.samples.append(chunk); self.masks.append(mchunk)
                
    def __len__(self): return len(self.samples)
        
    def __getitem__(self,idx):
        seq=self.samples[idx]; m=self.masks[idx]
        x=seq[:-1]; y=seq[1:]; m=m[1:]
        if len(x)<self.block_size:
            pad=self.block_size-len(x)
            x+=[self.tok.PAD]*pad; y+=[self.tok.PAD]*pad; m+=[False]*pad
        else:
            x=x[:self.block_size]; y=y[:self.block_size]; m=m[:self.block_size]

        # 손실 마스킹 : keep=False 위치의 타깃을 PAD로 덮어서 ignore_index에 걸리게 함
        y=[yy if keep else self.tok.PAD for yy,keep in zip(y,m)]
        return torch.tensor(x),torch.tensor(y)

[위치 정보 추가]

1. 기존 프로젝트
- sinusoidal positional encoding은 별도 전처리 없이 모델 forward에서 자동 생성했습니다.

2. 변경 내역
- 전처리 단계에서 시퀀스를 block_size 단위로 잘라주고, 모델 내부에서 learned positional embedding을 시퀀스 길이에 맞춰 자동으로 더했습니다.

### 3. GPT 모델 구성하기 

[전체 구조]

1. 기존 프로젝트
- Encoder와 Decoder 두 블록으로 구성
- Encoder는 입력 질문을 self-attention으로 인코딩, Decoder는 Encoder 출력을 cross-attention으로 참조하면서 답변을 생성

2. 변경 내역
- Decoder-only 구조 (Encoder 완전히 제거)
- Self-Attention만 사용하며 미래 토큰을 가리는 causal mask 적용
- Cross-attention이 없는 단일 LM 구조

In [10]:
class CausalSelfAttention(nn.Module):
    def __init__(self, d_model, n_heads, attn_dropout=0.0, resid_dropout=0.0):
        super().__init__()
        assert d_model % n_heads == 0
        self.n_heads = n_heads
        self.d_head = d_model // n_heads
        self.qkv = nn.Linear(d_model, 3*d_model, bias=False)
        self.proj = nn.Linear(d_model, d_model, bias=False)
        self.attn_drop = nn.Dropout(attn_dropout)
        self.resid_drop = nn.Dropout(resid_dropout)
        
    def forward(self,x):
        B,T,C = x.size()
        qkv = self.qkv(x)
        q,k,v = qkv.chunk(3,dim=-1)
        q = q.view(B,T,self.n_heads,self.d_head).transpose(1,2)
        k = k.view(B,T,self.n_heads,self.d_head).transpose(1,2)
        v = v.view(B,T,self.n_heads,self.d_head).transpose(1,2)
        att = (q @ k.transpose(-2,-1))/ (self.d_head**0.5)
        mask = torch.ones(T,T,device=x.device).triu(1)
        att = att.masked_fill(mask.bool(),float("-inf")).softmax(-1)
        att = self.attn_drop(att)
        y = att @ v
        y = y.transpose(1,2).contiguous().view(B,T,C)
        return self.resid_drop(self.proj(y))

In [11]:
class MLP(nn.Module):
    def __init__(self,d_model,d_ff,dropout=0.0):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model,d_ff),
            nn.GELU(),
            nn.Linear(d_ff,d_model),
            nn.Dropout(dropout),
        )
    def forward(self,x): return self.net(x)

class Block(nn.Module):
    def __init__(self,d_model,n_heads,d_ff,attn_dropout=0.0,resid_dropout=0.0):
        super().__init__()
        self.ln1=nn.LayerNorm(d_model)
        self.attn=CausalSelfAttention(d_model,n_heads,attn_dropout,resid_dropout)
        self.ln2=nn.LayerNorm(d_model)
        self.mlp=MLP(d_model,d_ff,resid_dropout)
        
    def forward(self,x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

In [12]:
class GPT1(nn.Module):
    def __init__(self,vocab_size,block_size,n_layer=4,n_head=8,d_model=256,d_ff=1024,
                 emb_dropout=0.1,resid_dropout=0.1):
        super().__init__()
        self.vocab_size=vocab_size
        self.block_size=block_size
        self.tok_emb=nn.Embedding(vocab_size,d_model)
        self.pos_emb=nn.Embedding(block_size,d_model)
        self.drop=nn.Dropout(emb_dropout)
        self.blocks=nn.ModuleList([Block(d_model,n_head,d_ff,0.1,resid_dropout) for _ in range(n_layer)])
        self.ln_f=nn.LayerNorm(d_model)
        self.lm_head=nn.Linear(d_model,vocab_size,bias=False)
        self.lm_head.weight=self.tok_emb.weight
        self.apply(self._init)
        
    def _init(self,m):
        if isinstance(m,(nn.Linear,nn.Embedding)):
            nn.init.normal_(m.weight,mean=0.0,std=0.02)
        if isinstance(m,nn.Linear) and m.bias is not None:
            nn.init.zeros_(m.bias)
            
    def forward(self,idx,targets=None):
        B,T=idx.size()
        assert T<=self.block_size
        pos=torch.arange(T,device=idx.device).unsqueeze(0) # Positional Encoding
        x=self.tok_emb(idx)+self.pos_emb(pos)
        x=self.drop(x)
        for blk in self.blocks: x=blk(x)
        x=self.ln_f(x)
        logits=self.lm_head(x)
        loss=None
        if targets is not None:
            PAD=getattr(tok,"PAD",0)
            loss=F.cross_entropy(logits.view(-1,logits.size(-1)),targets.view(-1),ignore_index=PAD)
        return logits,loss
        
    def summary_text(self):
        params=sum(p.numel() for p in self.parameters())
        return (
            f"GPT1(vocab={self.vocab_size},block={self.block_size},layers={len(self.blocks)})\\n"
            f"d_model={self.tok_emb.embedding_dim}, d_ff={self.blocks[0].mlp.net[0].out_features}\\n"
            f"params={params:,}"
        )

In [13]:
@dataclass
class HParams:
    block_size:int=256      # 한 시퀀스 최대 토큰 길이
    batch_size:int=32       # 학습 미니배치 크기
    n_layer:int=8           # Transformer Block 개수 (깊이)
    n_head:int=8            # Multi-Head Attention의 head 수
    d_model:int=512         # 임베딩 차원 (토큰 벡터 차원)
    d_ff:int=2048           # 피드포워드 레이어 차원 (확장된 차원)
    emb_dropout:float=0.1   # 임베딩 입력 직후 dropout 비율
    resid_dropout:float=0.1 # 블록 내 residual connection 뒤 dropout 비율
    lr:float=2e-4           # AdamW 학습률
    weight_decay:float=0.01 # AdamW 정규화 (가중치 감쇠)
    epochs:int=15           # 학습 epoch 수
    device:str=device       # "cuda" 또는 "cpu"
    
    def model_kwargs(self):
        return dict(block_size=self.block_size,n_layer=self.n_layer,n_head=self.n_head,
                    d_model=self.d_model,d_ff=self.d_ff,
                    emb_dropout=self.emb_dropout,resid_dropout=self.resid_dropout)

In [14]:
def apply_repetition_penalty(logits, prev_ids, penalty=1.1):
    if penalty==1.0 or prev_ids is None: return logits
    uniq=torch.unique(prev_ids)
    logits[:,uniq]/=penalty
    return logits

- ```generate``` : 추론 전용(no_grad) 생성 함수
- ```idx``` : 현재까지의 토큰(배치 1×길이)
- ```max_new_tokens``` : 새로 만들 최대 토큰 수
- ```temperature/top_p``` : 샘플링 제어
- ```repetition_penalty``` : 반복 억제

In [15]:
@torch.no_grad()
def generate(model, idx, max_new_tokens=64, temperature=0.8, top_p=0.9, repetition_penalty=1.1, stop_ids=None):
    model.eval()
    stop_ids=set(stop_ids or [])
    for _ in range(max_new_tokens):
        idx_cond=idx[:,-model.block_size:]
        logits,_=model(idx_cond)
        logits=logits[:,-1,:]
        logits=apply_repetition_penalty(logits,idx,penalty=repetition_penalty)
        logits=logits/temperature
        probs=torch.softmax(logits,dim=-1)
        sorted_probs,sorted_idx=torch.sort(probs,descending=True)
        cum=torch.cumsum(sorted_probs,dim=-1)
        mask=cum>top_p; mask[...,0]=False
        sorted_probs[mask]=0
        sorted_probs/=sorted_probs.sum(-1,keepdim=True)
        next_sorted=torch.multinomial(sorted_probs,1)
        next_id=sorted_idx.gather(-1,next_sorted)
        idx=torch.cat([idx,next_id],dim=1)
        token=int(next_id.item())
        if token in stop_ids or token==getattr(tok,"EOS",None):
            break
    return idx

### 4. 모델 평가하기

[출력 생성]

1. 기존 프로젝트
- 주로 Greedy search를 사용했었습니다.

2. 변경 내역
- 샘플링 기반 생성(temperature, top-p, reptition penalty, stop_ids)을 적용했습니다.
- 반복을 줄이고 다양한 응답을 생성할 수 있도록 했습니다.

In [16]:
# 데이터셋/로더 준비
h=HParams()
model=GPT1(vocab_size=tok.vocab_size,**h.model_kwargs()).to(h.device)
train_ds=DialogDataset(dialogs_train,tok,h.block_size,mask_user_loss=True)
val_ds=DialogDataset(dialogs_val,tok,h.block_size,mask_user_loss=True)
train_dl=torch.utils.data.DataLoader(train_ds,batch_size=h.batch_size,shuffle=True)
val_dl=torch.utils.data.DataLoader(val_ds,batch_size=h.batch_size)

print("=== model.summary ===")
print(model.summary_text())

opt=torch.optim.AdamW(model.parameters(),lr=h.lr,weight_decay=h.weight_decay)

for ep in range(1,h.epochs+1):
    model.train(); tot=0;n=0;t0=time.time()
    for xb,yb in train_dl:
        xb,yb=xb.to(h.device),yb.to(h.device)
        opt.zero_grad();_,loss=model(xb,yb)
        loss.backward(); opt.step()
        tot+=float(loss.item())*xb.size(0); n+=xb.size(0)
    tr_loss=tot/max(1,n)
    model.eval();vt=0;vn=0
    with torch.no_grad():
        for xb,yb in val_dl:
            xb,yb=xb.to(h.device),yb.to(h.device)
            _,vloss=model(xb,yb)
            vt+=float(vloss.item())*xb.size(0); vn+=xb.size(0)
    print(f"Epoch {ep}/{h.epochs} - {time.time()-t0:.1f}s - loss:{tr_loss:.4f} - val:{vt/max(1,vn):.4f}")

=== model.summary ===
GPT1(vocab=4003,block=256,layers=8)\nd_model=512, d_ff=2048\nparams=27,384,320
Epoch 1/15 - 82.8s - loss:5.3557 - val:4.9030
Epoch 2/15 - 82.5s - loss:4.7257 - val:4.5055
Epoch 3/15 - 82.6s - loss:4.3352 - val:4.2298
Epoch 4/15 - 82.7s - loss:4.0591 - val:4.0490
Epoch 5/15 - 82.6s - loss:3.8397 - val:3.9303
Epoch 6/15 - 82.6s - loss:3.6336 - val:3.8477
Epoch 7/15 - 82.4s - loss:3.4323 - val:3.7742
Epoch 8/15 - 82.5s - loss:3.2129 - val:3.7392
Epoch 9/15 - 82.5s - loss:2.9880 - val:3.6595
Epoch 10/15 - 82.6s - loss:2.7412 - val:3.6389
Epoch 11/15 - 82.8s - loss:2.4942 - val:3.6096
Epoch 12/15 - 82.8s - loss:2.2295 - val:3.6010
Epoch 13/15 - 82.8s - loss:1.9649 - val:3.5976
Epoch 14/15 - 82.8s - loss:1.6985 - val:3.5984
Epoch 15/15 - 82.7s - loss:1.4400 - val:3.5836


In [17]:
# 생성 테스트
q=val_df.iloc[0]["Q"]
ids=[getattr(tok,"BOS",0),getattr(tok,"USR",1)]+tok.encode_text(q,add_special=False)+[getattr(tok,"BOT",2)]
idx=torch.tensor([ids],dtype=torch.long,device=h.device)
out=generate(model,idx,max_new_tokens=80,temperature=0.8,top_p=0.9,repetition_penalty=1.1,
             stop_ids={getattr(tok,"USR",-1),getattr(tok,"DELIM",-1)})
print("Q:",q)
print("A:",tok.decode(out[0].tolist()).strip())

Q: 사랑이란 뭘까?
A: 사랑이란 뭘까? 직접 물어보세요. 차분히 당당하게 말해보세요. 그렇지 않아요. 무엇이 같이 들어질게요. 고민만 있다면 당신을 생각해보세요. 충분히 그럴게요. 힘들었을거라 생각해요. 내세요. 신경쓰지 마세요. 자연스러합니다. 기다리세요. 후회해졌하는 건 없어요. 있어요. 이별이었을? 생각해 볼게요. 좋아하는 기억으로 아파하게 표현해보는 것도 중요한 것 같네 도와주는
